In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime, timedelta
from itertools import combinations


import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # Cell의 모든 반환값 출력

# 기본 데이터 세팅

In [2]:
블록원데이터 = pd.DataFrame({"블록명":["S1", "S2", "S3", "S4"], 
                    "중량": [50, 60, 30, 20], 
                    "가로": [10, 10, 5, 5], 
                    "세로":[10, 12, 6, 6], 
                    "표준공기":[2, 3, 1, 2], 
                    "납기":["2024-02-15", "2024-02-15", "2024-02-15", "2024-02-15"],
                    "정반배치":[0, 0, 0, 0]})
블록원데이터.head()

,블록명,중량,가로,세로,표준공기,납기,정반배치
0,S1,50,10,10,2,2024-02-15,0
1,S2,60,10,12,3,2024-02-15,0
2,S3,30,5,6,1,2024-02-15,0
3,S4,20,5,6,2,2024-02-15,0


In [3]:
정반원데이터 = pd.DataFrame({"정반명":["A", "B", "C"],
                   "가능중량":[100, 100, 30],
                   "가로":[10, 10, 5],
                   "세로":[20, 10, 10],
                   })
정반원데이터.head()

,정반명,가능중량,가로,세로
0,A,100,10,20
1,B,100,10,10
2,C,30,5,10


In [4]:
정반원데이터.index[정반원데이터["정반명"]=="B"].values[0]

1

In [5]:
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 28)
날짜집합  = pd.date_range(start=start_date, end=end_date, freq='D')
날짜집합

DatetimeIndex(['2024-02-01', '2024-02-02', '2024-02-03', '2024-02-04',
               '2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08',
               '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12',
               '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16',
               '2024-02-17', '2024-02-18', '2024-02-19', '2024-02-20',
               '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-24',
               '2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28'],
              dtype='datetime64[ns]', freq='D')

In [6]:
블록집합 = 블록원데이터["블록명"].unique().tolist()
블록집합

['S1', 'S2', 'S3', 'S4']

In [7]:
최초정반집합 = 정반원데이터["정반명"].unique().tolist()
최초정반집합

['A', 'B', 'C']

In [8]:
쌩전체경우의수 = len(날짜집합) * len(블록집합) * len(최초정반집합)
쌩전체경우의수

336

# 블록 데이터 전처리 함수

In [9]:
# df apply 함수에 복수개의 변수를 던져주기
import functools
def unpack_df_columns(func):
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        series = args[0]
        return func(*series.values)
    return _unpack_df_columns

In [10]:
@unpack_df_columns
def 최소요구착수일구하기(납기, 공기):
    result = pd.to_datetime(납기) - timedelta(days=int(공기))
    return result.date()

In [11]:
착수일가중치, 공기가중치, 크기가중치 = 0.7, 0.5, 0.5

@unpack_df_columns
def 블록우선순위구하기(날순, 공순, 크순):
    global 착수일가중치, 공기가중치, 크기가중치
    result = np.round((날순*착수일가중치 + 공순*공기가중치 + 크순*크기가중치)/3,1)
    return result

@unpack_df_columns
def 길이리스트만들기(길이1, 길이2):
    result = []
    가로 = max(길이1, 길이2)  
    세로 = min(길이1, 길이2)
    result.append(가로)
    result.append(가로)
    result.append(세로)
    result.append(세로)
    return result

In [12]:
def 블록데이터전처리(블록원데이터):
    df1 = 블록원데이터.copy()
    df1["납기"] = pd.to_datetime(df1["납기"])
    df1["사이즈"] = df1.eval("가로*세로")
    df1["최소요구착수일"] = df1[["납기", "표준공기"]].apply(최소요구착수일구하기, axis=1)
    df1["날짜순서"] = df1["최소요구착수일"].rank()
    df1["공기순서"] = df1["표준공기"].rank(ascending=False)
    df1["크기순서"] = df1["사이즈"].rank(ascending=False)
    df1["우선순위"] = df1[["날짜순서", "공기순서", "크기순서"]].apply(블록우선순위구하기, axis=1)
    df1["길이리스트"] = df1[["가로", "세로"]].apply(길이리스트만들기, axis=1)
    df1["최장길이"] = df1["길이리스트"].apply(lambda x : max(x))
    df1["최소길이"] = df1["길이리스트"].apply(lambda x : min(x))
#     df1["면적리스트"] = df1["길이리스트"].apply(면적리스트구하기)
    df1 = df1.drop(['가로', '세로'], axis=1)
    df1 = df1.sort_values(by=["우선순위"])
    return df1

In [13]:
df1 = 블록데이터전처리(블록원데이터)
df1

,블록명,중량,표준공기,납기,정반배치,사이즈,최소요구착수일,날짜순서,공기순서,크기순서,우선순위,길이리스트,최장길이,최소길이
1,S2,60,3,2024-02-15,0,120,2024-02-12,1.0,1.0,1.0,0.6,"[12, 12, 10, 10]",12,10
0,S1,50,2,2024-02-15,0,100,2024-02-13,2.5,2.5,2.0,1.3,"[10, 10, 10, 10]",10,10
3,S4,20,2,2024-02-15,0,30,2024-02-13,2.5,2.5,3.5,1.6,"[6, 6, 5, 5]",6,5
2,S3,30,1,2024-02-15,0,30,2024-02-14,4.0,4.0,3.5,2.2,"[6, 6, 5, 5]",6,5


# 정반 데이터 전처리 함수

In [14]:
중량가중치, 크기가중치 = 0.5, 0.7

@unpack_df_columns
def 정반우선순위구하기(중순, 크순):
    global 중량가중치, 크기가중치
    result = np.round((중순*중량가중치 + 크순*크기가중치)/3,1)
    return result

def 면적리스트구하기(길이리스트):
    면적리스트 = []
    if 길이리스트[0] == 길이리스트[1]:
        면적리스트.append(길이리스트[0]*길이리스트[3])
    else:
        면적리스트  = []
        면적리스트.append(길이리스트[0]*길이리스트[3])
        면적리스트.append(길이리스트[1]*길이리스트[2])
    return 면적리스트

In [15]:
def 정반데이터전처리(정반원데이터):
    df2 = 정반원데이터.copy()
    df2["사이즈"] = df2.eval("가로 * 세로")
    df2["중량순서"] = df2["가능중량"].rank(ascending=False)
    df2["크기순서"] = df2["사이즈"].rank(ascending=False)
    df2["우선순위"] = df2[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
    df2["길이리스트"] = df2[["가로", "세로"]].apply(길이리스트만들기, axis=1)
    df2["최장길이"] = df2["길이리스트"].apply(lambda x : max(x))
    df2["최소길이"] = df2["길이리스트"].apply(lambda x : min(x))
    df2["면적리스트"] = df2["길이리스트"].apply(면적리스트구하기)
    df2 = df2.drop(['가로', '세로'], axis=1)
    df2 = df2.sort_values(by=["우선순위"])
    return df2

In [16]:
df2 = 정반데이터전처리(정반원데이터)
df2

,정반명,가능중량,사이즈,중량순서,크기순서,우선순위,길이리스트,최장길이,최소길이,면적리스트
0,A,100,200,1.5,1.0,0.5,"[20, 20, 10, 10]",20,10,[200]
1,B,100,100,1.5,2.0,0.7,"[10, 10, 10, 10]",10,10,[100]
2,C,30,50,3.0,3.0,1.2,"[10, 10, 5, 5]",10,5,[50]


In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   정반명     3 non-null      object 
 1   가능중량    3 non-null      int64  
 2   사이즈     3 non-null      int64  
 3   중량순서    3 non-null      float64
 4   크기순서    3 non-null      float64
 5   우선순위    3 non-null      float64
 6   길이리스트   3 non-null      object 
 7   최장길이    3 non-null      int64  
 8   최소길이    3 non-null      int64  
 9   면적리스트   3 non-null      object 
dtypes: float64(3), int64(4), object(3)
memory usage: 264.0+ bytes


# 배치달력 함수

In [18]:
def create_init_calendar(시작년:int, 시작월:int, 종료년:int, 종료월:int, 정반집합):
    start_date = datetime(시작년, 시작월, 1)
    end_date = datetime(종료년, 종료월, 28)
    날짜집합  = pd.date_range(start=start_date, end=end_date, freq='D')
    
    배치달력 = pd.DataFrame()
    배치달력.index = 날짜집합
   
    for 정반 in 정반집합:
        배치달력[f"{정반}"] = 0

    return 배치달력

In [19]:
최초정반집합 = 정반원데이터["정반명"].unique().tolist()
배치달력 = create_init_calendar(2024, 2, 2024, 2, 최초정반집합)
배치달력

,A,B,C
2024-02-01,0,0,0
2024-02-02,0,0,0
2024-02-03,0,0,0
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


In [20]:
def update_배치달력(배치달력, 정반명, 착수날짜, 필요공기, 정반집합):
    
    신규칼럼리스트 = 정반집합.copy()
    try:
        for 현칼럼 in 배치달력.columns:
            신규칼럼리스트.remove(현칼럼)

        for 신규칼럼 in 신규칼럼리스트:
            # 신규칼럼에는 모정반의 배치달력 정보를 카피(모정반의 제약조건을 승계하도록)
            배치달력[f"{신규칼럼}"] = 배치달력[f"{신규칼럼[:1]}"]

        시점인덱스 = list(배치달력.index.strftime('%Y-%m-%d')).index(착수날짜)
        배치달력[f"{정반명}"].iloc[시점인덱스:시점인덱스+필요공기] = 1
        return 배치달력
    
    except:
        return 배치달력

# 공기달력 함수

In [21]:
def create_공기달력(배치달력, 날짜집합, 정반집합):
    total_list = []

    for 정반 in 정반집합:
        검토대상 = 배치달력[f"{정반}"].tolist()

        new_list = []
        new_num = 0
        for idx, i in enumerate(검토대상):
            if i == 0:
                new_num = new_num  + 1
                new_list.append(new_num)
            else:
                new_list.append(0)
                new_num = 0
        total_list.append(new_list)
        
    new_total = []
    for original_list in total_list:

        result_list = []
        group = []
        for num in original_list:
            if num == 0 and group:
                result_list.extend(reversed(group))
                group = []
            group.append(num)

        result_list.extend(reversed(group))

        new_total.append(result_list)

    공기달력 = pd.DataFrame()
    공기달력.index = 날짜집합

    for idx, 정반 in enumerate(정반집합):
        공기달력[f"{정반}"] =  new_total[idx]

    for 정반 in 정반집합:
        if 공기달력[f"{정반}"][0]== 0:
            공기달력[f"{정반}"] = 공기달력[f"{정반}"].shift(1)
        else:
            pass
    공기달력.fillna(0, inplace=True)
    return 공기달력

# 공백순서달력(first_zero) 함수

In [22]:
def create_공백순서달력(배치_달력, 정반집합, 날짜집합):
    total = []

    for 정반 in 배치_달력.columns.tolist():
        
        input_list = 배치_달력[f"{정반}"].tolist()
        
        counter = 1
        result_list = []

        for idx, x in enumerate(input_list):

            if idx == 0:
                if x == 1:
                    result_list.append(0)
                else:
                    result_list.append(counter)
                    counter += 1

            else:   
                if input_list[idx-1] == 1 and x == 0:
                    result_list.append(counter)
                    counter += 1
                else:
                    result_list.append(0)

        total.append(result_list)

    공백순서달력 = pd.DataFrame()
    공백순서달력.index = 날짜집합

    for idx, 정반 in enumerate(배치_달력.columns.tolist()):
        공백순서달력[f"{정반}"] =  total[idx]

    return 공백순서달력

# 착수가능일찾기 함수

In [23]:
def 착수가능일찾기(공기달력, 공백순서달력, 정반, 표준공기):
        
    first_zeros = []
    
    for idx, i in enumerate(공백순서달력[f"{정반}"].tolist()):
        if i != 0:
            first_zeros.append(공백순서달력.index.strftime('%Y-%m-%d').values[idx])

    for idx, 착수가능일 in enumerate(first_zeros):
        
        
        착수가능일인덱스 = list(공기달력.index.strftime('%Y-%m-%d')).index(first_zeros[idx])
        착수가능일의확보가능공기 = 공기달력[f"{정반}"].iloc[착수가능일인덱스]
        
        if 착수가능일의확보가능공기 > 표준공기:
            
            return 착수가능일
        
        else:
            pass

# 간트차트 함수

In [24]:
import plotly.express as px
def draw_gant(df):
    fig = px.timeline(
        
        df, 
        x_start="착수일", 
        x_end="종료일", 
        y="정반명",
        color="정반구분",
        hover_data = ["표준공기", "조립중량"],
        text = "차트텍스트",
        opacity=0.7
        )

    fig.update_yaxes(autorange="reversed")          #if not specified as 'reversed', the tasks will be listed from bottom up       
    fig.update_layout(
                    # title='Project Plan Gantt Chart',
                    hoverlabel_bgcolor='#DAEEED',   #Change the hover tooltip background color to a universal light blue color. If not specified, the background color will vary by team or completion pct, depending on what view the user chooses
                    bargap=0.2,
                    height=500,              
                    xaxis_title="", 
                    yaxis_title="정반명", 
                    font=dict(
                        family="Courier New, monospace",
                        size=12,  # Set the font size here
                        color="RebeccaPurple"
                        ),
    )
    fig.show()

# 달력생성 및 업데이트 연습

## 최초달력생성

In [83]:
배치달력 = create_init_calendar(2024, 2, 2024, 2, 최초정반집합)
배치달력.iloc[:2,:] = 1   ## 날짜 계산의 편의를 위해 초반 2일을 배치상태로 설정
배치달력

,A,B,C
2024-02-01,1,1,1
2024-02-02,1,1,1
2024-02-03,0,0,0
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


In [84]:
정반집합 = 배치달력. columns.tolist()
공기달력 = create_공기달력(배치달력, 날짜집합, 정반집합)
공기달력

,A,B,C
2024-02-01,0.0,0.0,0.0
2024-02-02,0.0,0.0,0.0
2024-02-03,26.0,26.0,26.0
2024-02-04,25.0,25.0,25.0
2024-02-05,24.0,24.0,24.0
2024-02-06,23.0,23.0,23.0
2024-02-07,22.0,22.0,22.0
2024-02-08,21.0,21.0,21.0
2024-02-09,20.0,20.0,20.0
2024-02-10,19.0,19.0,19.0


In [85]:
공백순서달력 = create_공백순서달력(배치달력, 정반집합, 날짜집합)
공백순서달력

,A,B,C
2024-02-01,0,0,0
2024-02-02,0,0,0
2024-02-03,1,1,1
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


## 블록배치

In [86]:
착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, "A", 13)
착수가능일

'2024-02-03'

## 달력 업데이트

In [87]:
새정반집합 = ["A","B","C","A_추가"]
배치달력 = update_배치달력(배치달력, "A", "2024-02-10", 2, 새정반집합)
배치달력

,A,B,C,A_추가
2024-02-01,1,1,1,1
2024-02-02,1,1,1,1
2024-02-03,0,0,0,0
2024-02-04,0,0,0,0
2024-02-05,0,0,0,0
2024-02-06,0,0,0,0
2024-02-07,0,0,0,0
2024-02-08,0,0,0,0
2024-02-09,0,0,0,0
2024-02-10,1,0,0,0


In [30]:
공기달력 = create_공기달력(배치달력, 날짜집합, 새정반집합)
공기달력

,A,B,C,A_추가
2024-02-01,0.0,0.0,0.0,0.0
2024-02-02,0.0,0.0,0.0,0.0
2024-02-03,7.0,26.0,26.0,26.0
2024-02-04,6.0,25.0,25.0,25.0
2024-02-05,5.0,24.0,24.0,24.0
2024-02-06,4.0,23.0,23.0,23.0
2024-02-07,3.0,22.0,22.0,22.0
2024-02-08,2.0,21.0,21.0,21.0
2024-02-09,1.0,20.0,20.0,20.0
2024-02-10,0.0,19.0,19.0,19.0


In [31]:
공백순서달력 = create_공백순서달력(배치달력, 새정반집합, 날짜집합)
공백순서달력

,A,B,C,A_추가
2024-02-01,0,0,0,0
2024-02-02,0,0,0,0
2024-02-03,1,1,1,1
2024-02-04,0,0,0,0
2024-02-05,0,0,0,0
2024-02-06,0,0,0,0
2024-02-07,0,0,0,0
2024-02-08,0,0,0,0
2024-02-09,0,0,0,0
2024-02-10,0,0,0,0


# 물리적 스펙 적합도 함수

In [32]:
# def 면적_최장길이적합도_검토(정반면적, 정반최장길이, 블록면적, 블록최장길이):
#     if 정반면적 >= 블록면적 and 정반최장길이 >= 블록최장길이:
#         return "적합"
#     else:
#         return "부적합"

In [33]:
def 면적_최장길이적합도_검토1(정반면적리스트, 정반길이리스트, 블록면적, 블록길이리스트):
    result = []
    블록최장길이 = max(블록길이리스트)
    블록최소길이 = min(블록길이리스트)
    for i in range(len(정반면적리스트)):
        if i == 0:
            if 정반면적리스트[i] >= 블록면적 and max(정반길이리스트[0], 정반길이리스트[3]) >= 블록최장길이 and min(정반길이리스트[0], 정반길이리스트[3]) >= 블록최소길이:
                result.append("적합")
            else:
                result.append("부적합")
        
        else:
            if 정반면적리스트[i] >= 블록면적 and max(정반길이리스트[1], 정반길이리스트[2]) >= 블록최장길이 and min(정반길이리스트[1], 정반길이리스트[2]) >= 블록최소길이:
                result.append("적합")
            else:
                result.append("부적합")
                
    return result

In [34]:
면적리스트 = [32, 30]
정반길이리스트 = [8, 10, 3, 4]
블록면적 = 30
블록길이리스트 = [6, 6, 5, 5]
면적_최장길이적합도_검토1(면적리스트, 정반길이리스트, 블록면적, 블록길이리스트)

['부적합', '부적합']

In [35]:
면적리스트 = [200]
정반길이리스트 = [20, 20, 10, 10]
블록면적 = 100
블록길이리스트 = [10, 10, 10, 10]
면적_최장길이적합도_검토1(면적리스트, 정반길이리스트, 블록면적, 블록길이리스트)

['적합']

In [36]:
def 배치후잔여정반길이리스트(정반길이리스트, 블록길이리스트):
    
    블록최소길이 = min(블록길이리스트)
    블록최장길이 = max(블록길이리스트)
    정반최소길이 = min(정반길이리스트)
    정반최장길이 = max(정반길이리스트)
    
    if 정반최소길이 == 블록최소길이 and 정반최장길이 >= 블록최장길이:
        정반길이리스트[0] = max(정반길이리스트[0] - 블록최장길이,0)
        정반길이리스트[1] = max(정반길이리스트[1] - 블록최장길이,0)
        정반길이리스트[2] = 블록최소길이
        정반길이리스트[3] = 블록최소길이

    elif 정반최장길이 == 블록최장길이 and 정반최소길이 >= 블록최소길이:
        정반길이리스트[0] = 블록최장길이
        정반길이리스트[1] = 블록최장길이
        정반길이리스트[2] = max(정반길이리스트[2] - 블록최소길이,0)
        정반길이리스트[3] = max(정반길이리스트[3] - 블록최소길이,0)

    else:
        정반길이리스트[0] = max(정반길이리스트[0] - 블록최장길이,0)
        정반길이리스트[2] = max(정반길이리스트[2] - 블록최소길이,0)
    
    정반길이리스트 = [max([정반길이리스트[0], 정반길이리스트[3]]),
              max([정반길이리스트[1], 정반길이리스트[2]]),
              min([정반길이리스트[1], 정반길이리스트[2]]),
              min([정반길이리스트[0], 정반길이리스트[3]])]
    
    return 정반길이리스트
    

In [37]:
정반길이리스트 = [8, 8, 10, 10]
블록길이리스트 = [6, 6, 5, 5]
결과 = 배치후잔여정반길이리스트(정반길이리스트, 블록길이리스트)
결과

[10, 8, 5, 2]

In [38]:
정반길이리스트 = [10, 10, 8, 8]
블록길이리스트 = [6, 6, 5, 5]
결과 = 배치후잔여정반길이리스트(정반길이리스트, 블록길이리스트)
결과

[8, 10, 3, 4]

In [39]:
def 면적리스트구하기(길이리스트):
    면적리스트 = []
    if 길이리스트[0] == 길이리스트[1]:
        면적리스트.append(길이리스트[0]*길이리스트[3])
    else:
        면적리스트  = []
        면적리스트.append(길이리스트[0]*길이리스트[3])
        면적리스트.append(길이리스트[1]*길이리스트[2])
    return 면적리스트

# 생산계획수립함수

In [40]:
def 생산계획수립(블록데이터, 정반데이터, 배치달력, 정반쪼개는면적_Thresh):

    # 결과모음리스트
    배정된블록 = []
    배정된정반 = []
    착수일 = []
    표준공기 = []
    종료일 = []
    조립중량  = []
    
    자식정반고유번호 = 0
        
    수정블록리스트 = 블록데이터["블록명"].tolist()
    수정정반리스트 = 정반데이터["정반명"].tolist()

    
    공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
    공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
    
    for _ in tqdm(range(len(수정블록리스트))):
        
        if 수정블록리스트:
            target_block = 수정블록리스트[0]
        else:
            print("수정블록리스트에 검토대상 잔여블록이 없습니다.")
            print()
            break
            
        blk_index = 블록데이터[블록데이터["블록명"]==target_block].index.values[0]
        target_weight = 블록데이터[블록데이터["블록명"]==target_block]["중량"].values[0]
        target_size = 블록데이터[블록데이터["블록명"]==target_block]["사이즈"].values[0]
        least_start_date = 블록데이터[블록데이터["블록명"]==target_block]["최소요구착수일"].values[0]
        target_표준공기 = 블록데이터[블록데이터["블록명"]==target_block]["표준공기"].values[0]
        target_길이리스트 = 블록데이터[블록데이터["블록명"]==target_block]["길이리스트"].values[0]
        target_최장길이 = 블록데이터[블록데이터["블록명"]==target_block]["최장길이"].values[0]
        target_최소길이 = 블록데이터[블록데이터["블록명"]==target_block]["최소길이"].values[0]
        
        print()
        print(f"*** 타겟블록-name:{target_block},weight:{target_weight},size:{target_size},길이리스트:{target_길이리스트},최장길이:{target_최장길이},최소길이:{target_최소길이},최소착수요구일:{least_start_date},표준공기:{target_표준공기}")
        print()
        
        
        가능정반_dict = {}
        for 정반 in 수정정반리스트:
            

            
            공백순서1인덱스 = list(공백순서달력[f"{정반}"]).index(1)
            공백순서1인덱스의날짜 = 배치달력.index[공백순서1인덱스]
            공백순서1인덱스의날짜의확보가능공기 = 공기달력[f"{정반}"][공백순서1인덱스]
            공백순서1인덱스날짜의공백순서 = 공백순서달력[f"{정반}"][공백순서1인덱스]
            weight_capa = 정반데이터[정반데이터["정반명"]==정반]["가능중량"].values[0]
            size_capa = 정반데이터[정반데이터["정반명"]==정반]["사이즈"].values[0]
            
            try:
                공백순서2인덱스 = list(공백순서달력[f"{정반}"]).index(2)
                공백순서2인덱스의날짜 = 배치달력.index[공백순서2인덱스]
                공백순서2인덱스의날짜의확보가능공기 = 공기달력[f"{정반}"][공백순서2인덱스]
                공백순서2인덱스날짜의공백순서 = 공백순서달력[f"{정반}"][공백순서2인덱스]
                
                if 공백순서1인덱스의날짜 <= least_start_date and 공백순서1인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서1인덱스날짜의공백순서 == 1:
                    가능정반_dict[정반] = 공백순서1인덱스의날짜
                elif 공백순서2인덱스의날짜 <= least_start_date and 공백순서2인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서2인덱스날짜의공백순서 == 2:
                    가능정반_dict[정반] = 공백순서2인덱스의날짜
                else:
                    pass
                
            except:
                if 공백순서1인덱스의날짜 <= least_start_date and 공백순서1인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서1인덱스날짜의공백순서 == 1:
                    가능정반_dict[정반] = 공백순서1인덱스의날짜
                else:
                    pass
                pass
    
        if 가능정반_dict != {}:
            print(f"*** 가능정반 :{가능정반_dict}")
            최선조기착수가능정반 = [key for key, value in 가능정반_dict.items() if value == min(가능정반_dict.values())]   # 여러개일수 있으므로 리스트로 반환
            print(f"*** 최선조기착수 가능정반:{min(가능정반_dict.values())} ---> {최선조기착수가능정반}")  
            print()
            
            ### 물리적 스펙 가능 검토 #########
            물리적스펙가능정반 = []
            for 후보정반 in 최선조기착수가능정반:
                
                ## 엄마정반 스펙 #########################################3
                weight_capa = 정반데이터[정반데이터["정반명"]==후보정반]["가능중량"].values[0]
                정반면적 = 정반데이터[정반데이터["정반명"]==후보정반]["사이즈"].values[0]  
                정반최장길이 = 정반데이터[정반데이터["정반명"]==후보정반]["최장길이"].values[0]
                정반길이리스트 = 정반데이터[정반데이터["정반명"]==후보정반]["길이리스트"].values[0]
                정반면적리스트 = 정반데이터[정반데이터["정반명"]==후보정반]["면적리스트"].values[0]
                
                ## 물리적 적합도 검토 ##############################
                print(f">>>{후보정반}정반의 면적_최장길이 적합도 함수 체크")
                print(f">>>정반면적리스트:{정반면적리스트}, 정반길이리스트:{정반길이리스트}, 블록사이즈:{target_size}, 블록최장길이:{target_최장길이}")
                
                물리적적합도리스트 = 면적_최장길이적합도_검토1(정반면적리스트, 정반길이리스트, target_size, target_길이리스트)
                print(f">>>정반 {후보정반}의 면적/사이즈 배치가능 여부:{물리적적합도리스트} / 정반가능중량:{weight_capa} 정반면적:{정반면적}, 길이리스트:{정반길이리스트}, 정반최장길이:{정반최장길이}")
                print()
                
                if "적합" in 물리적적합도리스트:
                    물리적스펙가능정반.append(후보정반)
                else:
                    pass
            
            print(f"*** 물리적 스펙 가능 정반: {물리적스펙가능정반}")
            print()
            
            if 물리적스펙가능정반:
                
                랜덤최선정반 = random.choice(물리적스펙가능정반)
                랜덤정반인덱스 = 정반데이터.index[정반데이터["정반명"]==랜덤최선정반].values[0]
                
                랜덤정반최장길이 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["최장길이"].values[0]
                랜덤정반길이리스트 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["길이리스트"].values[0]
                랜덤정반면적리스트 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["면적리스트"].values[0]
                랜덤정반면적 = min(랜덤정반면적리스트)

                잔여면적 = 랜덤정반면적 - target_size
                잔여면적비율 = 잔여면적 / 랜덤정반면적
                print(f"*** 랜덤선택최선정반:{랜덤최선정반}")
                print(f"*** 랜덤정반스펙- 길이리스트:{랜덤정반길이리스트}, 면적리스트:{랜덤정반면적리스트}, MIN면적:{랜덤정반면적}, block_size:{target_size}, 잔여면적:{잔여면적}, 잔여면적비율:{np.round(잔여면적비율,2)}")
                print()
                
                if 잔여면적비율 >= 정반쪼개는면적_Thresh:
                    
                    새정반이름 = 랜덤최선정반+f"_{자식정반고유번호}"
                    자식정반고유번호 += 1
                    새정반면적 = 정반면적 * 잔여면적비율
                    기존정반새면적 = 정반면적 - 새정반면적
                    가능정반인덱스 = 정반데이터[정반데이터["정반명"]==랜덤최선정반].index.values[0]
                    정반길이리스트 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["길이리스트"].values[0]
                    
                    ### 분할 자식정반 스펙 계산 ################3
                    
                    원정반길이리스트 = target_길이리스트
                    원정반면적리스트 = 면적리스트구하기(원정반길이리스트)
                    새정반길이리스트 = 배치후잔여정반길이리스트(정반길이리스트, target_길이리스트)
                    
                    정반데이터.at[랜덤정반인덱스, "길이리스트"] = 원정반길이리스트
                    정반데이터.at[랜덤정반인덱스, "면적리스트"] = 원정반면적리스트
                    
      
                    새정반최장길이 = max(새정반길이리스트)
                    새정반최소길이 = min(새정반길이리스트)
                    새정반면적리스트 = 면적리스트구하기(새정반길이리스트)
                    
                    정반데이터.loc[가능정반인덱스,"사이즈"] = 기존정반새면적
                    정반데이터.loc[len(정반데이터)] = {"정반명":새정반이름, "가능중량": weight_capa, 
                                             "사이즈":새정반면적, "길이리스트":새정반길이리스트, "최장길이":새정반최장길이, 
                                             "최소길이":새정반최소길이, "면적리스트":새정반면적리스트}

                    정반데이터["중량순서"] = 정반데이터["가능중량"].rank(ascending=False)
                    정반데이터["크기순서"] = 정반데이터["사이즈"].rank(ascending=False)
                    정반데이터["우선순위"] = 정반데이터[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
                    정반데이터 = 정반데이터.sort_values(by=["우선순위"])
                    정반데이터["정반구분"] = 정반데이터["정반명"].apply(lambda x: x[:1])

                    print(f">>>잔여면적비율 {np.round(잔여면적비율,1)*100}%로 30% 이상이므로 정반 쪼개기 - 자식정반이름 :{새정반이름} / 자식정반면적:{np.round(새정반면적,1)} / 엄마정반면적: {np.round(기존정반새면적,1)}") 
                    print(f"*** 원정반길이리스트: {target_길이리스트}")
                    print(f"*** 새정반길이리스트: {새정반길이리스트}")
                    print()
                    수정정반리스트.append(새정반이름)  
                    
                else:
                    print(f">>>1순위 정반의 잔여면적비율({np.round(잔여면적비율,1)}이 Thresh({정반쪼개는면적_Thresh}) 비율보다 작아 쪼갤 수 없습니다.")
                    print()
                    
                착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, 랜덤최선정반, target_표준공기)
                블록데이터.loc[blk_index, "정반배치"] = 1

                배정결과 = {"블록명": target_block, "정반명": 랜덤최선정반, "착수일": 착수가능일}   
                print(f"*** 최종배정결과 : {배정결과}") 
                print()

                배정된블록.append(target_block)
                배정된정반.append(랜덤최선정반)
                착수일.append(착수가능일)
                표준공기.append(target_표준공기)
                original_date = datetime.strptime(착수가능일, "%Y-%m-%d")
                종료날짜 = original_date + timedelta(days=int(target_표준공기)) 
                종료날짜 = 종료날짜.strftime("%Y-%m-%d")
                종료일.append(종료날짜)
                조립중량.append(target_weight)

                print(f"(블록배치전) 수정블록리스트 : {수정블록리스트}, 수정정반리스트 : {수정정반리스트}")
                수정블록리스트.remove(target_block) 
                print(f"(블록배치후) 수정블록리스트 : {수정블록리스트}, 수정정반리스트 : {수정정반리스트}")
                print("="*50)
                


                
                
                
                ##### 달력 1차 업데이트 ############
                배치달력 =  update_배치달력(배치달력, 랜덤최선정반, 착수가능일, target_표준공기, 수정정반리스트) 
                #배치달력.iloc[:5,:] = 1    ## 갱신시 하드코딩 2일차까지 강제로 1 채우기
                공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
                공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
                
                
                '''
                여기서 모정반, 자정반 병합 가능 여부 체크후 가능하면 병합하여 수정정반리스트 변경 필요
                '''
            
                
                수정정반리스트
                ##### 달력 2차 업데이트 ############
                배치달력 =  update_배치달력(배치달력, 랜덤최선정반, 착수가능일, target_표준공기, 수정정반리스트) 
                #배치달력.iloc[:5,:] = 1    ## 갱신시 하드코딩 2일차까지 강제로 1 채우기
                공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
                공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
                
                
            else:
                print("★ 물리적 스펙상 배치가능한 정반이 없습니다.!!!!")
                print("="*50)
                수정블록리스트.remove(target_block)
                pass
        else:
            print("★ 날짜 조건에 따른 배치가능한 정반이 없습니다.!!!!")
            print("="*50)
            수정블록리스트.remove(target_block)
            pass

    최종배정결과 = pd.DataFrame({"블록명":배정된블록, "정반명":배정된정반, "착수일":착수일, "표준공기":표준공기, "종료일": 종료일, "조립중량": 조립중량})
    최종배정결과["정반구분"] = 최종배정결과["정반명"].apply(lambda x: x[:1])
    return 최종배정결과, 블록데이터, 정반데이터, 배치달력, 공기달력, 공백순서달력

# 배치 시뮬레이션

In [62]:
data_num = 1

블록원데이터 = pd.read_excel(f"./data/data{data_num}.xlsx", sheet_name="블록데이터")
정반원데이터 = pd.read_excel(f"./data/data{data_num}.xlsx", sheet_name="정반데이터")

블록데이터 = 블록데이터전처리(블록원데이터)
정반데이터 = 정반데이터전처리(정반원데이터)
최초정반집합 = 정반원데이터["정반명"].unique().tolist()

배치달력 = create_init_calendar(2024, 2, 2024, 2, 최초정반집합) 
배치달력.iloc[:2,:] = 1   ## 날짜 계산의 편의를 위해 초반 2일을 배치상태로 설정

result = 생산계획수립(블록데이터, 정반데이터, 배치달력, 0.3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 29.20it/s]


*** 타겟블록-name:S2,weight:60,size:120,길이리스트:[12, 12, 10, 10],최장길이:12,최소길이:10,최소착수요구일:2024-02-12,표준공기:3

*** 가능정반 :{'A': Timestamp('2024-02-03 00:00:00', freq='D'), 'B': Timestamp('2024-02-03 00:00:00', freq='D')}
*** 최선조기착수 가능정반:2024-02-03 00:00:00 ---> ['A', 'B']

>>>A정반의 면적_최장길이 적합도 함수 체크
>>>정반면적리스트:[200], 정반길이리스트:[20, 20, 10, 10], 블록사이즈:120, 블록최장길이:12
>>>정반 A의 면적/사이즈 배치가능 여부:['적합'] / 정반가능중량:100 정반면적:200, 길이리스트:[20, 20, 10, 10], 정반최장길이:20

>>>B정반의 면적_최장길이 적합도 함수 체크
>>>정반면적리스트:[150], 정반길이리스트:[15, 15, 10, 10], 블록사이즈:120, 블록최장길이:12
>>>정반 B의 면적/사이즈 배치가능 여부:['적합'] / 정반가능중량:100 정반면적:150, 길이리스트:[15, 15, 10, 10], 정반최장길이:15

*** 물리적 스펙 가능 정반: ['A', 'B']

*** 랜덤선택최선정반:B
*** 랜덤정반스펙- 길이리스트:[15, 15, 10, 10], 면적리스트:[150], MIN면적:150, block_size:120, 잔여면적:30, 잔여면적비율:0.2

>>>1순위 정반의 잔여면적비율(0.2이 Thresh(0.3) 비율보다 작아 쪼갤 수 없습니다.

*** 최종배정결과 : {'블록명': 'S2', '정반명': 'B', '착수일': '2024-02-03'}

(블록배치전) 수정블록리스트 : ['S2', 'S1', 'S4', 'S3'], 수정정반리스트 : ['A', 'B', 'C']
(블록배치후) 수정블록리스트 : ['S1', 'S4', 'S3'], 수정정반리스트 :

In [63]:
배치결과 = result[0]
배치결과 = 배치결과[["블록명", "정반구분", "정반명", "착수일", "표준공기", "종료일", "조립중량"]].sort_values(by="정반구분")
배치결과

,블록명,정반구분,정반명,착수일,표준공기,종료일,조립중량
1,S1,A,A,2024-02-03,2,2024-02-05,50
3,S3,A,A_0,2024-02-03,1,2024-02-04,30
0,S2,B,B,2024-02-03,3,2024-02-06,60
2,S4,C,C,2024-02-03,2,2024-02-05,20


In [64]:
@unpack_df_columns
def create_text(블록명, 표준공기):
    return str(블록명)+" / "+"조립중량: "+str(표준공기)+"ton"

In [65]:
배치결과["차트텍스트"] = 배치결과[["블록명","표준공기"]].apply(create_text, axis=1)
배치결과

,블록명,정반구분,정반명,착수일,표준공기,종료일,조립중량,차트텍스트
1,S1,A,A,2024-02-03,2,2024-02-05,50,S1 / 조립중량: 2ton
3,S3,A,A_0,2024-02-03,1,2024-02-04,30,S3 / 조립중량: 1ton
0,S2,B,B,2024-02-03,3,2024-02-06,60,S2 / 조립중량: 3ton
2,S4,C,C,2024-02-03,2,2024-02-05,20,S4 / 조립중량: 2ton


In [66]:
draw_gant(배치결과)

In [67]:
새정반리스트 = result[2].sort_values(by="정반구분")
새정반리스트 = 새정반리스트[["정반구분", "정반명", "가능중량", "사이즈", "중량순서", "크기순서", 
                 "우선순위", "길이리스트", "최장길이", "최소길이", "면적리스트"]]
새정반리스트

,정반구분,정반명,가능중량,사이즈,중량순서,크기순서,우선순위,길이리스트,최장길이,최소길이,면적리스트
0,A,A,100,100.0,3.0,2.0,1.0,"[10, 10, 10, 10]",20,10,[100]
5,A,A_0_2,100,28.0,3.0,5.0,1.7,"[10, 10, 5, 4]",10,4,[40]
3,A,A_0,100,12.0,3.0,6.0,1.9,"[6, 6, 5, 5]",10,10,[30]
1,B,B,100,150.0,3.0,1.0,0.7,"[15, 15, 10, 10]",15,10,[150]
4,C,C_1,100,40.0,3.0,4.0,1.4,"[5, 5, 4, 4]",5,4,[20]
2,C,C,30,60.0,6.0,3.0,1.7,"[6, 6, 5, 5]",10,5,[30]


In [68]:
배치후블록데이터 = result[1]
배치후블록데이터

,블록명,중량,표준공기,납기,정반배치,사이즈,최소요구착수일,날짜순서,공기순서,크기순서,우선순위,길이리스트,최장길이,최소길이
1,S2,60,3,2024-02-15,1,120,2024-02-12,1.0,1.0,1.0,0.6,"[12, 12, 10, 10]",12,10
0,S1,50,2,2024-02-15,1,100,2024-02-13,2.5,2.5,2.0,1.5,"[10, 10, 10, 10]",10,10
3,S4,20,2,2024-02-15,1,30,2024-02-13,2.5,2.5,3.5,1.8,"[6, 6, 5, 5]",6,5
2,S3,30,1,2024-02-15,1,30,2024-02-14,4.0,4.0,3.5,2.4,"[6, 6, 5, 5]",6,5


In [48]:
배치후배치달력 = result[3]
배치후배치달력

,A,B,C,A_0,A_0_1,C_2
2024-02-01,1,1,1,1,1,1
2024-02-02,1,1,1,1,1,1
2024-02-03,1,1,1,1,1,0
2024-02-04,1,1,1,1,1,0
2024-02-05,1,1,1,1,1,0
2024-02-06,1,1,1,1,1,0
2024-02-07,1,1,0,1,1,0
2024-02-08,1,1,0,1,1,0
2024-02-09,1,1,0,1,0,0
2024-02-10,1,1,0,0,0,0


In [49]:
배치후공기달력 = result[4]
배치후공기달력

,A,B,C,A_0,A_0_1,C_2
2024-02-01,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-02,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-03,0.0,0.0,0.0,0.0,0.0,26.0
2024-02-04,0.0,0.0,0.0,0.0,0.0,25.0
2024-02-05,0.0,0.0,0.0,0.0,0.0,24.0
2024-02-06,0.0,0.0,0.0,0.0,0.0,23.0
2024-02-07,0.0,0.0,22.0,0.0,0.0,22.0
2024-02-08,0.0,0.0,21.0,0.0,0.0,21.0
2024-02-09,0.0,0.0,20.0,0.0,20.0,20.0
2024-02-10,0.0,0.0,19.0,19.0,19.0,19.0


# 자식정반 병합후 수정정반리스트 업데이트

In [57]:
배치가능일 = "2024-02-12"
배치후배치달력[배치후배치달력.index==배치가능일]

,A,B,C,A_0,A_0_1,C_2
2024-02-12,1,1,0,0,0,0


In [51]:
def 배치달력_공기달력상태체크(배치달력, 날짜, 정반1, 정반2):
    # 특정일에 배치달력 및 공기달력 상태가 동일하면 병합처리하려고...
    정반1배치달력값 = 배치달력.at[날짜,정반1]
    정반1공기달력값 = 배치달력.at[날짜,정반1]
    
    정반2배치달력값 = 배치달력.at[날짜,정반2]
    정반2공기달력값 = 배치달력.at[날짜,정반2]
    
    if 정반1배치달력값 == 정반2배치달력값 and 정반1공기달력값 == 정반2공기달력값:
        return 1  # 병합가능
    else:
        return 0  # 병합불가능

In [60]:
배치달력_공기달력상태체크(배치후배치달력, "2024-02-12", "A", "A_0")

0

In [53]:
from itertools import combinations

def 자식정반병합후삭제(배치달력, 날짜):
    병합후삭제정반 = []
    정반구분리스트 = []
    for 정반명 in 배치달력.columns:
        정반구분리스트.append(정반명[:1])
    정반구분세트리스트 = list(set(정반구분리스트))


    for 정반구분 in 정반구분세트리스트:
        임시칼럼 = []
        for 정반명 in 배치달력.columns:

            if 정반구분 in 정반명:
                임시칼럼.append(정반명)
        combi = list(combinations(임시칼럼,2))
        print(f"콤비네이션: {combi}")  

        try:
            병합대상 = []
            for i in list(combi):
                print(f"병합후보: {i}")
                체크 = 배치달력_공기달력상태체크(배치달력, 날짜, i[0], i[1])
                if 체크: 
                    병합대상.append(i[0])
                    병합대상.append(i[1])
                else:
                    print("병합불가1")
                    pass

            병합대상 = list(set(병합대상))
            병합대상.sort()
            print(f"병합대상: {병합대상}")        
            삭제대상 = 병합대상[1:]
            print(f"삭제대상: {삭제대상}")
            for 삭제 in 삭제대상:
                병합후삭제정반.append(삭제)
            print("="*50)

        except:
            print("병합불가2")
            pass

    return 병합후삭제정반

In [61]:
자식정반병합후삭제(배치후배치달력, "2024-02-12")

콤비네이션: [('C', 'C_2')]
병합후보: ('C', 'C_2')
병합대상: ['C', 'C_2']
삭제대상: ['C_2']
콤비네이션: [('A', 'A_0'), ('A', 'A_0_1'), ('A_0', 'A_0_1')]
병합후보: ('A', 'A_0')
병합불가1
병합후보: ('A', 'A_0_1')
병합불가1
병합후보: ('A_0', 'A_0_1')
병합대상: ['A_0', 'A_0_1']
삭제대상: ['A_0_1']
콤비네이션: []
병합대상: []
삭제대상: []


['C_2', 'A_0_1']

In [55]:
def 리스트간중복원소제거(lst1, lst2):
    temp3 = []
    for i in lst1:
        if i not in lst2:
            temp3.append(i)
    return temp3

In [56]:
lst1 = ['A', 'A_0', 'B', 'A_0_1', 'C', 'C_2']
lst2 = ['A_0', 'A_0_1', 'C_2']
리스트간중복원소제거(lst1, lst2)

['A', 'B', 'C']

# 정반 통합 로직 반영 생산계획 수립

In [81]:
def 생산계획수립(블록데이터, 정반데이터, 배치달력, 정반쪼개는면적_Thresh):

    # 결과모음리스트
    배정된블록 = []
    배정된정반 = []
    착수일 = []
    표준공기 = []
    종료일 = []
    조립중량  = []
    
    자식정반고유번호 = 0
        
    수정블록리스트 = 블록데이터["블록명"].tolist()
    수정정반리스트 = 정반데이터["정반명"].tolist()

    
    공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
    공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
    
    for _ in tqdm(range(len(수정블록리스트))):
        
        if 수정블록리스트:
            target_block = 수정블록리스트[0]
        else:
            print("수정블록리스트에 검토대상 잔여블록이 없습니다.")
            print()
            break
            
        blk_index = 블록데이터[블록데이터["블록명"]==target_block].index.values[0]
        target_weight = 블록데이터[블록데이터["블록명"]==target_block]["중량"].values[0]
        target_size = 블록데이터[블록데이터["블록명"]==target_block]["사이즈"].values[0]
        least_start_date = 블록데이터[블록데이터["블록명"]==target_block]["최소요구착수일"].values[0]
        target_표준공기 = 블록데이터[블록데이터["블록명"]==target_block]["표준공기"].values[0]
        target_길이리스트 = 블록데이터[블록데이터["블록명"]==target_block]["길이리스트"].values[0]
        target_최장길이 = 블록데이터[블록데이터["블록명"]==target_block]["최장길이"].values[0]
        target_최소길이 = 블록데이터[블록데이터["블록명"]==target_block]["최소길이"].values[0]
        
        print()
        print(f"*** 타겟블록-name:{target_block},weight:{target_weight},size:{target_size},길이리스트:{target_길이리스트},최장길이:{target_최장길이},최소길이:{target_최소길이},최소착수요구일:{least_start_date},표준공기:{target_표준공기}")
        print()
        
        
        가능정반_dict = {}
        for 정반 in 수정정반리스트:
            

            
            공백순서1인덱스 = list(공백순서달력[f"{정반}"]).index(1)
            공백순서1인덱스의날짜 = 배치달력.index[공백순서1인덱스]
            공백순서1인덱스의날짜의확보가능공기 = 공기달력[f"{정반}"][공백순서1인덱스]
            공백순서1인덱스날짜의공백순서 = 공백순서달력[f"{정반}"][공백순서1인덱스]
            weight_capa = 정반데이터[정반데이터["정반명"]==정반]["가능중량"].values[0]
            size_capa = 정반데이터[정반데이터["정반명"]==정반]["사이즈"].values[0]
            
            try:
                공백순서2인덱스 = list(공백순서달력[f"{정반}"]).index(2)
                공백순서2인덱스의날짜 = 배치달력.index[공백순서2인덱스]
                공백순서2인덱스의날짜의확보가능공기 = 공기달력[f"{정반}"][공백순서2인덱스]
                공백순서2인덱스날짜의공백순서 = 공백순서달력[f"{정반}"][공백순서2인덱스]
                
                if 공백순서1인덱스의날짜 <= least_start_date and 공백순서1인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서1인덱스날짜의공백순서 == 1:
                    가능정반_dict[정반] = 공백순서1인덱스의날짜
                elif 공백순서2인덱스의날짜 <= least_start_date and 공백순서2인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서2인덱스날짜의공백순서 == 2:
                    가능정반_dict[정반] = 공백순서2인덱스의날짜
                else:
                    pass
                
            except:
                if 공백순서1인덱스의날짜 <= least_start_date and 공백순서1인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서1인덱스날짜의공백순서 == 1:
                    가능정반_dict[정반] = 공백순서1인덱스의날짜
                else:
                    pass
                pass
    
        if 가능정반_dict != {}:
            print(f"*** 가능정반 :{가능정반_dict}")
            최선조기착수가능정반 = [key for key, value in 가능정반_dict.items() if value == min(가능정반_dict.values())]   # 여러개일수 있으므로 리스트로 반환
            print(f"*** 최선조기착수 가능정반:{min(가능정반_dict.values())} ---> {최선조기착수가능정반}")  
            print()
            
            ### 물리적 스펙 가능 검토 #########
            물리적스펙가능정반 = []
            for 후보정반 in 최선조기착수가능정반:
                
                ## 엄마정반 스펙 #########################################3
                weight_capa = 정반데이터[정반데이터["정반명"]==후보정반]["가능중량"].values[0]
                정반면적 = 정반데이터[정반데이터["정반명"]==후보정반]["사이즈"].values[0]  
                정반최장길이 = 정반데이터[정반데이터["정반명"]==후보정반]["최장길이"].values[0]
                정반길이리스트 = 정반데이터[정반데이터["정반명"]==후보정반]["길이리스트"].values[0]
                정반면적리스트 = 정반데이터[정반데이터["정반명"]==후보정반]["면적리스트"].values[0]
                
                ## 물리적 적합도 검토 ##############################
                print(f">>>{후보정반}정반의 면적_최장길이 적합도 함수 체크")
                print(f">>>정반면적리스트:{정반면적리스트}, 정반길이리스트:{정반길이리스트}, 블록사이즈:{target_size}, 블록최장길이:{target_최장길이}")
                
                물리적적합도리스트 = 면적_최장길이적합도_검토1(정반면적리스트, 정반길이리스트, target_size, target_길이리스트)
                print(f">>>정반 {후보정반}의 면적/사이즈 배치가능 여부:{물리적적합도리스트} / 정반가능중량:{weight_capa} 정반면적:{정반면적}, 길이리스트:{정반길이리스트}, 정반최장길이:{정반최장길이}")
                print()
                
                if "적합" in 물리적적합도리스트:
                    물리적스펙가능정반.append(후보정반)
                else:
                    pass
            
            print(f"*** 물리적 스펙 가능 정반: {물리적스펙가능정반}")
            print()
            
            if 물리적스펙가능정반:
                
                랜덤최선정반 = random.choice(물리적스펙가능정반)
                랜덤정반인덱스 = 정반데이터.index[정반데이터["정반명"]==랜덤최선정반].values[0]
                
                랜덤정반최장길이 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["최장길이"].values[0]
                랜덤정반길이리스트 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["길이리스트"].values[0]
                랜덤정반면적리스트 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["면적리스트"].values[0]
                랜덤정반면적 = min(랜덤정반면적리스트)

                잔여면적 = 랜덤정반면적 - target_size
                잔여면적비율 = 잔여면적 / 랜덤정반면적
                print(f"*** 랜덤선택최선정반:{랜덤최선정반}")
                print(f"*** 랜덤정반스펙- 길이리스트:{랜덤정반길이리스트}, 면적리스트:{랜덤정반면적리스트}, MIN면적:{랜덤정반면적}, block_size:{target_size}, 잔여면적:{잔여면적}, 잔여면적비율:{np.round(잔여면적비율,2)}")
                print()
                
                if 잔여면적비율 >= 정반쪼개는면적_Thresh:
                    
                    새정반이름 = 랜덤최선정반+f"_{자식정반고유번호}"
                    자식정반고유번호 += 1
                    새정반면적 = 정반면적 * 잔여면적비율
                    기존정반새면적 = 정반면적 - 새정반면적
                    가능정반인덱스 = 정반데이터[정반데이터["정반명"]==랜덤최선정반].index.values[0]
                    정반길이리스트 = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["길이리스트"].values[0]
                    
                    ### 분할 자식정반 스펙 계산 ################3
                    
                    원정반길이리스트 = target_길이리스트
                    원정반면적리스트 = 면적리스트구하기(원정반길이리스트)
                    새정반길이리스트 = 배치후잔여정반길이리스트(정반길이리스트, target_길이리스트)
                    
                    정반데이터.at[랜덤정반인덱스, "길이리스트"] = 원정반길이리스트
                    정반데이터.at[랜덤정반인덱스, "면적리스트"] = 원정반면적리스트
                    
      
                    새정반최장길이 = max(새정반길이리스트)
                    새정반최소길이 = min(새정반길이리스트)
                    새정반면적리스트 = 면적리스트구하기(새정반길이리스트)
                    
                    정반데이터.loc[가능정반인덱스,"사이즈"] = 기존정반새면적
                    정반데이터.loc[len(정반데이터)] = {"정반명":새정반이름, "가능중량": weight_capa, 
                                             "사이즈":새정반면적, "길이리스트":새정반길이리스트, "최장길이":새정반최장길이, 
                                             "최소길이":새정반최소길이, "면적리스트":새정반면적리스트}

                    정반데이터["중량순서"] = 정반데이터["가능중량"].rank(ascending=False)
                    정반데이터["크기순서"] = 정반데이터["사이즈"].rank(ascending=False)
                    정반데이터["우선순위"] = 정반데이터[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
                    정반데이터 = 정반데이터.sort_values(by=["우선순위"])
                    정반데이터["정반구분"] = 정반데이터["정반명"].apply(lambda x: x[:1])

                    print(f">>>잔여면적비율 {np.round(잔여면적비율,1)*100}%로 30% 이상이므로 정반 쪼개기 - 자식정반이름 :{새정반이름} / 자식정반면적:{np.round(새정반면적,1)} / 엄마정반면적: {np.round(기존정반새면적,1)}") 
                    print(f"*** 원정반길이리스트: {target_길이리스트}")
                    print(f"*** 새정반길이리스트: {새정반길이리스트}")
                    print()
                    수정정반리스트.append(새정반이름)  
                    
                else:
                    print(f">>>1순위 정반의 잔여면적비율({np.round(잔여면적비율,1)}이 Thresh({정반쪼개는면적_Thresh}) 비율보다 작아 쪼갤 수 없습니다.")
                    print()
                    
                착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, 랜덤최선정반, target_표준공기)
                블록데이터.loc[blk_index, "정반배치"] = 1

                배정결과 = {"블록명": target_block, "정반명": 랜덤최선정반, "착수일": 착수가능일}   
                print(f"*** 최종배정결과 : {배정결과}") 
                print()

                배정된블록.append(target_block)
                배정된정반.append(랜덤최선정반)
                착수일.append(착수가능일)
                표준공기.append(target_표준공기)
                original_date = datetime.strptime(착수가능일, "%Y-%m-%d")
                종료날짜 = original_date + timedelta(days=int(target_표준공기)) 
                종료날짜 = 종료날짜.strftime("%Y-%m-%d")
                종료일.append(종료날짜)
                조립중량.append(target_weight)

                print(f"(블록배치전) 수정블록리스트 : {수정블록리스트}, 수정정반리스트 : {수정정반리스트}")
                수정블록리스트.remove(target_block) 
                print(f"(블록배치후) 수정블록리스트 : {수정블록리스트}, 수정정반리스트 : {수정정반리스트}")
                print("="*50)
                


                
                
                
                ##### 달력 1차 업데이트 ############
                배치달력 =  update_배치달력(배치달력, 랜덤최선정반, 착수가능일, target_표준공기, 수정정반리스트) 
                #배치달력.iloc[:5,:] = 1    ## 갱신시 하드코딩 2일차까지 강제로 1 채우기
                공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
                공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
                
                
                '''
                여기서 모정반, 자정반 병합 가능 여부 체크후 가능하면 병합하여 수정정반리스트 변경 필요
                '''
                삭제대상정반 = 자식정반병합후삭제(배치달력, 착수가능일)
                print(f"삭제대상정반 : {삭제대상정반}")
            
                
                수정정반리스트1 = 리스트간중복원소제거(수정정반리스트, 삭제대상정반)
                print(f"삭제할놈 삭제후 수정정반리스트 : {수정정반리스트1}")
                
                print(f"(블록배치후) 수정블록리스트1 : {수정블록리스트}, 수정정반리스트1 : {수정정반리스트1}")
                
                ##### 달력 2차 업데이트 ############
                배치달력1 =  update_배치달력(배치달력, 랜덤최선정반, 착수가능일, target_표준공기, 수정정반리스트1) 
                공기달력1 = create_공기달력(배치달력1, 날짜집합, 수정정반리스트)
                공백순서달력1 = create_공백순서달력(배치달력1, 수정정반리스트, 날짜집합)
                
                
            else:
                print("★ 물리적 스펙상 배치가능한 정반이 없습니다.!!!!")
                print("="*50)
                수정블록리스트.remove(target_block)
                pass
        else:
            print("★ 날짜 조건에 따른 배치가능한 정반이 없습니다.!!!!")
            print("="*50)
            수정블록리스트.remove(target_block)
            pass

    최종배정결과 = pd.DataFrame({"블록명":배정된블록, "정반명":배정된정반, "착수일":착수일, "표준공기":표준공기, "종료일": 종료일, "조립중량": 조립중량})
    최종배정결과["정반구분"] = 최종배정결과["정반명"].apply(lambda x: x[:1])
    return 최종배정결과, 블록데이터, 정반데이터, 배치달력1, 공기달력1, 공백순서달력1

In [82]:
data_num = 1

블록원데이터 = pd.read_excel(f"./data/data{data_num}.xlsx", sheet_name="블록데이터")
정반원데이터 = pd.read_excel(f"./data/data{data_num}.xlsx", sheet_name="정반데이터")

블록데이터 = 블록데이터전처리(블록원데이터)
정반데이터 = 정반데이터전처리(정반원데이터)
최초정반집합 = 정반원데이터["정반명"].unique().tolist()

배치달력 = create_init_calendar(2024, 2, 2024, 2, 최초정반집합) 
배치달력.iloc[:2,:] = 1   ## 날짜 계산의 편의를 위해 초반 2일을 배치상태로 설정

result = 생산계획수립(블록데이터, 정반데이터, 배치달력, 0.3)

 50%|█████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                 | 2/4 [00:00<00:00, 13.41it/s]


*** 타겟블록-name:S2,weight:60,size:120,길이리스트:[12, 12, 10, 10],최장길이:12,최소길이:10,최소착수요구일:2024-02-12,표준공기:3

*** 가능정반 :{'A': Timestamp('2024-02-03 00:00:00', freq='D'), 'B': Timestamp('2024-02-03 00:00:00', freq='D')}
*** 최선조기착수 가능정반:2024-02-03 00:00:00 ---> ['A', 'B']

>>>A정반의 면적_최장길이 적합도 함수 체크
>>>정반면적리스트:[200], 정반길이리스트:[20, 20, 10, 10], 블록사이즈:120, 블록최장길이:12
>>>정반 A의 면적/사이즈 배치가능 여부:['적합'] / 정반가능중량:100 정반면적:200, 길이리스트:[20, 20, 10, 10], 정반최장길이:20

>>>B정반의 면적_최장길이 적합도 함수 체크
>>>정반면적리스트:[150], 정반길이리스트:[15, 15, 10, 10], 블록사이즈:120, 블록최장길이:12
>>>정반 B의 면적/사이즈 배치가능 여부:['적합'] / 정반가능중량:100 정반면적:150, 길이리스트:[15, 15, 10, 10], 정반최장길이:15

*** 물리적 스펙 가능 정반: ['A', 'B']

*** 랜덤선택최선정반:A
*** 랜덤정반스펙- 길이리스트:[20, 20, 10, 10], 면적리스트:[200], MIN면적:200, block_size:120, 잔여면적:80, 잔여면적비율:0.4

>>>잔여면적비율 40.0%로 30% 이상이므로 정반 쪼개기 - 자식정반이름 :A_0 / 자식정반면적:60.0 / 엄마정반면적: 90.0
*** 원정반길이리스트: [12, 12, 10, 10]
*** 새정반길이리스트: [10, 10, 8, 8]

*** 최종배정결과 : {'블록명': 'S2', '정반명': 'A', '착수일': '2024-02-03'}

(블록배치전) 수정블록리스트 : ['S2', 'S1', 'S4

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 13.01it/s]

>>>잔여면적비율 40.0%로 30% 이상이므로 정반 쪼개기 - 자식정반이름 :B_1_3 / 자식정반면적:20.0 / 엄마정반면적: 30.0
*** 원정반길이리스트: [6, 6, 5, 5]
*** 새정반길이리스트: [5, 5, 4, 4]

*** 최종배정결과 : {'블록명': 'S3', '정반명': 'B_1', '착수일': '2024-02-03'}

(블록배치전) 수정블록리스트 : ['S3'], 수정정반리스트 : ['A', 'B', 'C', 'A_0', 'B_1', 'C_2', 'B_1_3']
(블록배치후) 수정블록리스트 : [], 수정정반리스트 : ['A', 'B', 'C', 'A_0', 'B_1', 'C_2', 'B_1_3']
콤비네이션: [('C', 'C_2')]
병합후보: ('C', 'C_2')
병합불가1
병합대상: []
삭제대상: []
콤비네이션: [('A', 'A_0')]
병합후보: ('A', 'A_0')
병합불가1
병합대상: []
삭제대상: []
콤비네이션: [('B', 'B_1'), ('B', 'B_1_3'), ('B_1', 'B_1_3')]
병합후보: ('B', 'B_1')
병합후보: ('B', 'B_1_3')
병합후보: ('B_1', 'B_1_3')
병합대상: ['B', 'B_1', 'B_1_3']
삭제대상: ['B_1', 'B_1_3']
삭제대상정반 : ['B_1', 'B_1_3']
삭제할놈 삭제후 수정정반리스트 : ['A', 'B', 'C', 'A_0', 'C_2']
(블록배치후) 수정블록리스트1 : [], 수정정반리스트1 : ['A', 'B', 'C', 'A_0', 'C_2']


In [78]:
result[2]

,정반명,가능중량,사이즈,중량순서,크기순서,우선순위,길이리스트,최장길이,최소길이,면적리스트,정반구분
1,B,100,150.0,3.0,1.0,0.7,"[15, 15, 10, 10]",15,10,[150],B
0,A,100,100.0,3.0,2.0,1.0,"[10, 10, 10, 10]",20,10,[100],A
4,C_1,100,40.0,3.0,4.0,1.4,"[5, 5, 4, 4]",5,4,[20],C
2,C,30,60.0,6.0,3.0,1.7,"[6, 6, 5, 5]",10,5,[30],C
5,A_0_2,100,28.0,3.0,5.0,1.7,"[10, 10, 5, 4]",10,4,[40],A
3,A_0,100,12.0,3.0,6.0,1.9,"[6, 6, 5, 5]",10,10,[30],A
